# Installation

1. Install [bottle](https://bottlepy.org)
<pre>
pip install bottle
</pre>

1. Install [lorem](https://pypi.python.org/pypi/lorem)
<pre>
pip install lorem
</pre>

1. Install mysqlclient
<pre>
pip install mysqlclient
</pre>

# Git clone

<pre>
git clone 
</pre>

# Writing code

Open you favorite editor. Go to your source code directory.

1. Create new package `blog'
1. Inside the `blog` create a new catalogue `views`
1. In `views` create new file `index.tpl` with the following content

4. Create new module `server.py`

<pre>
import bottle

@bottle.route('/')
def index():
    return bottle.template('index')
    
bottle.debug(True)
bottle.run(host='localhost', port=8086)  # Start the webserver running and wait for requests
</pre>

Run the server.

Open a browser and go to http://localhost:8086

If you see `Template 'index' not found.` see here: [https://bottlepy.org/docs/dev/faq.html](https://bottlepy.org/docs/dev/faq.html) 

## Adding CSS style

Download [bootstrap source code](http://getbootstrap.com/getting-started/#download)

In `blog` directory create `static` directory. 
Extract bootstrap.zip. Copy `js`, `css` and `fonts` folders to the `static` directory.

Create `style.css` file in `static/css` directory.

In `index.tpl` in the head section add 
<pre>
&lt;link rel="stylesheet" type="text/css" href="/static/css/style.css"&gt;
</pre>

Run the server. Note that you will see 
<pre>
"GET /static/css/style.css HTTP/1.1" 404 736
</pre>

In the server.py add a function that will serve static content.
<pre>
import os

@bottle.route('/static/&lt;directory&gt;/&lt;filename&gt;')
def server_static(directory, filename):
    root = os.path.join(os.getcwd(), 'static', directory)
    return bottle.static_file(filename, root=root)
</pre>

Take bootstrap. Go to examples/blog. Copy the contents of the css file and paste into style.css we have. Now fix it. 

You can also copy the contents of blog.html (put it on github), that is a smaller version of the example/blog/index.html

## Navigation

Within server.py write navigation between pages (draw an image on Adrian's surface pro)

<pre>
@bottle.route('/')
def index():
    return bottle.template('index')
    
@bottle.route('/about')
def about():
    return bottle.template('about')
    
@bottle.get('/blog')
def blog():
    return bottle.template('blog')
    
@bottle.get('/post')
def new_entry():
    return bottle.template('post')
    
@bottle.post('/blog')
def add_entry():
    # do stuff
    return blog()
</pre>



Copy base.tpl and blog.tpl to your project

Create remaining tpl files: post.tpl and about.tpl. Leave them blank for now.

In blog.tpl add a button that allows to add a new entry:
<pre>
&lt;div class="blog-pagination"&gt;
    &lt;a class="btn btn-primary" href="/post"&gt;Add new entry&lt;/a&gt;
&lt;/div&gt;
</pre>

In post.tpl paste the following code:
<pre>
% rebase('base.tpl')
&lt;div class="col-sm-8 blog-main"&gt;
    &lt;form id="new_post" action="blog" method="post"&gt;
        &lt;div class="blog-post"&gt;
        &lt;/div&gt;
        &lt;div class="blog-pagination"&gt;
            &lt;button class="btn btn-primary" type="submit" id="submit"&gt;Save&lt;/button&gt;
        &lt;/div&gt;
    &lt;/form&gt;
&lt;/div&gt;
</pre>

Change blog() function to give us some posts
<pre>
@bottle.get('/blog')
def blog():
    import lorem
    import random 
    from datetime import date
    posts = []
    today = date.today().toordinal()    
    for i in range(10):
        paragraph = lorem.text()
        post_date = date.fromordinal(random.randint(0, today))
        title = lorem.sentence()
        post_dict = {'post': paragraph, 'date' : post_date, 'author': 'agata', 'title': title}
        posts.append(post_dict)
    posts = sorted(posts, key=lambda post : post['date'])
    return bottle.template('blog', posts=posts)
</pre>

Update blog.tpl to display our random posts.

<pre>
%for idx, post in enumerate(posts):
    &lt;div class="blog-post"&gt;
        &lt;h2 class="blog-post-title"&gt;{{post['title']}}&lt;/h2&gt;
        &lt;p class="blog-post-meta"&gt;{{post['date']}} by &lt;a href="#"&gt;{{post['author']}}&lt;/a&gt;&lt;/p&gt;
        &lt;p&gt;{{post['post']}}&lt;/p&gt;
    &lt;/div&gt;
%end
</pre>

We can also create a master page for our web-pages. Create base.tpl and place all the html content that will always be there. For the content that will be changing put
<pre>
{{!base}}
</pre>

Copy the contents from the blog.tpl and put the above tag where necessary.

Create new file blog2.tpl. Put 
<pre>
% rebase('base.tpl')
</pre>
at the top of the file. Below paste the content of blog.tpl 

In base.tpl change the top navigation to:
<pre>
&lt;nav class="nav blog-nav"&gt;
    % if page is None:
        &lt;a class="nav-link active" href="/"&gt;Home&lt;/a&gt;
    % else:
        &lt;a class="nav-link" href="/"&gt;Home&lt;/a&gt;
    % end
    % if page == 'blog':
        &lt;a class="nav-link active" href="/blog"&gt;Blog&lt;/a&gt;
    % else:
        &lt;a class="nav-link" href="/blog"&gt;Blog&lt;/a&gt;
    % end
    % if page == 'post':
        &lt;a class="nav-link active" href="/post"&gt;New post&lt;/a&gt;
    % else:
        &lt;a class="nav-link" href="/post"&gt;New post&lt;/a&gt;
    % end
    % if page == 'about':
        &lt;a class="nav-link active" href="/about"&gt;About&lt;/a&gt;
    % else:
        &lt;a class="nav-link" href="/about"&gt;About&lt;/a&gt;
    % end
&lt;/nav>
</pre>

Create

Now we have to change the server.py to send the page data. 
<pre>    
@bottle.route('/about')
def about():
    return bottle.template('about', page='about')
    
@bottle.get('/blog')
def blog():
    # previously existing code, do not change
    return bottle.template('blog2', posts=posts, page='blog')
    
@bottle.get('/post')
def new_entry():
    return bottle.template('post', page='post')
    
</pre>

# Adding a new post

At the bottom of style.css add
<pre>
/*
 * New post form
 */
.form-new-post {
  padding: 15px;
}
.form-new-post .form-control{
  margin-bottom: 10px;
}
</pre>

and in the post.tpl paste
<pre>
% rebase('base.tpl')
&lt;form id="new_post" action="blog" method="post" class="form-new-post"&gt;
    &lt;h2 class="form-heading"&gt;New Message&lt;/h2&gt;
    &lt;label for="title"&gt;Title:&lt;/label&gt;
    &lt;input type="text" class="form-control" name="title" id="title"/&gt;
    &lt;label for="author"&gt;Author:&lt;/label&gt;
    &lt;input type="text" class="form-control" name="author" id="author"/&gt;
    &lt;label for"content"&gt;Content:&lt;/label&gt;
    &lt;textarea name="content" class="form-control" id="content"&gt;&lt;/textarea&gt;
    &lt;button class="btn btn-primary" type="submit" id="submit"&gt;Save&lt;/button&gt;
&lt;/form&gt;
</pre>

Update the server to collect the data from the form
<pre>
@bottle.post('/blog')
def add_entry():
    title = bottle.request.forms.get('title')
    author = bottle.request.forms.get('author')
    content = bottle.request.forms.get('content')
    post_date = date.today()
    print('Post "{}"\n by {} added on {}\nContent: {}'.format(title, author, post_date, content))
    return blog()
</pre>